# Google Sheets Integration for AI Oracle Project

This notebook demonstrates how to integrate Google Sheets with the AI Oracle project for data management, script storage, and analytics tracking.

## Prerequisites

1. **Google Cloud Project Setup:**
   - Go to [Google Cloud Console](https://console.cloud.google.com/)
   - Create a new project or select an existing one
   - Enable the Google Sheets API
   - Enable the Google Drive API

2. **Service Account Credentials:**
   - Navigate to "IAM & Admin" > "Service Accounts"
   - Create a service account
   - Download the JSON credentials file
   - Save it as `credentials/google_sheets_credentials.json`
   - Share your Google Sheet with the service account email

3. **Environment Variables:**
   - Add to your `.env` file:
   ```
   GOOGLE_SHEETS_CREDENTIALS_PATH=credentials/google_sheets_credentials.json
   GOOGLE_SHEET_ID=your_sheet_id_here
   ```

## Use Cases for AI Oracle
- Store video scripts and metadata
- Track content performance metrics
- Manage ARG storyline elements
- Coordinate multi-channel content calendars

## 1. Import Required Libraries

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

print("✅ All libraries imported successfully")

## 2. Setup Authentication

This section handles Google Sheets API authentication using service account credentials.

In [ ]:
# Load environment variables
load_dotenv()

# Define the scope for Google Sheets and Drive access
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

def authenticate_gspread():
    """
    Authenticate with Google Sheets API using service account credentials.
    
    Returns:
        gspread.Client: Authenticated gspread client
    """
    try:
        # Get credentials path from environment or use default
        creds_path = os.getenv(
            'GOOGLE_SHEETS_CREDENTIALS_PATH',
            'credentials/google_sheets_credentials.json'
        )
        
        # Check if credentials file exists
        if not os.path.exists(creds_path):
            raise FileNotFoundError(
                f"Credentials file not found at: {creds_path}\n"
                "Please follow the setup instructions in the Prerequisites section."
            )
        
        # Authenticate using service account
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            creds_path, SCOPES
        )
        client = gspread.authorize(credentials)
        
        print("✅ Successfully authenticated with Google Sheets API")
        return client
    
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
        return None
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        print("Please check your credentials file and ensure it's valid JSON.")
        return None

# Authenticate
client = authenticate_gspread()

## 3. Reading Data from Google Sheets

Examples of how to read data from Google Sheets and convert it to pandas DataFrames for analysis.

In [ ]:
def read_sheet_to_dataframe(client, sheet_id, worksheet_name='Sheet1'):
    """
    Read data from a Google Sheet and convert to pandas DataFrame.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID (from URL)
        worksheet_name: Name of the worksheet to read
    
    Returns:
        pd.DataFrame: Data from the sheet
    """
    try:
        # Open the spreadsheet
        spreadsheet = client.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)
        
        # Get all values from the worksheet
        data = worksheet.get_all_records()
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        print(f"✅ Successfully read {len(df)} rows from '{worksheet_name}'")
        return df
    
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"❌ Error: Spreadsheet not found. Check the sheet ID and sharing permissions.")
        return None
    except gspread.exceptions.WorksheetNotFound:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found.")
        return None
    except Exception as e:
        print(f"❌ Error reading sheet: {e}")
        return None

# Example: Read data from a sheet
if client:
    # Get sheet ID from environment or use a placeholder
    sheet_id = os.getenv('GOOGLE_SHEET_ID', 'YOUR_SHEET_ID_HERE')
    
    if sheet_id != 'YOUR_SHEET_ID_HERE':
        df = read_sheet_to_dataframe(client, sheet_id, 'Scripts')
        if df is not None:
            print("\nFirst 5 rows:")
            print(df.head())
    else:
        print("⚠️  Please set GOOGLE_SHEET_ID in your .env file")
else:
    print("⚠️  Skipping read operation - client not authenticated")

## 4. Writing Data to Google Sheets

Examples of how to write and update data in Google Sheets from pandas DataFrames.

In [ ]:
def write_dataframe_to_sheet(client, sheet_id, df, worksheet_name='Data', mode='replace'):
    """
    Write a pandas DataFrame to a Google Sheet.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        df: pandas DataFrame to write
        worksheet_name: Name of the worksheet
        mode: 'replace' to overwrite, 'append' to add rows
    
    Returns:
        bool: True if successful
    """
    try:
        spreadsheet = client.open_by_key(sheet_id)
        
        # Try to get existing worksheet or create new one
        try:
            worksheet = spreadsheet.worksheet(worksheet_name)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(
                title=worksheet_name,
                rows=str(len(df) + 100),
                cols=str(len(df.columns))
            )
            print(f"📝 Created new worksheet '{worksheet_name}'")
        
        if mode == 'replace':
            # Clear existing data and write new data
            worksheet.clear()
            # Update with header and data
            worksheet.update([df.columns.values.tolist()] + df.values.tolist())
            print(f"✅ Replaced data in '{worksheet_name}' with {len(df)} rows")
        
        elif mode == 'append':
            # Append data to existing sheet
            worksheet.append_rows(df.values.tolist())
            print(f"✅ Appended {len(df)} rows to '{worksheet_name}'")
        
        return True
    
    except Exception as e:
        print(f"❌ Error writing to sheet: {e}")
        return False

# Example: Create sample data and write to sheet
if client:
    # Create sample AI Oracle script tracking data
    sample_data = pd.DataFrame({
        'Script_ID': ['SCR001', 'SCR002', 'SCR003'],
        'Title': [
            'The Oracle Awakens',
            'Hidden Messages in the Code',
            'The Simulation Theory'
        ],
        'Status': ['Completed', 'In Progress', 'Draft'],
        'Word_Count': [1500, 1200, 800],
        'Created_Date': [datetime.now().strftime('%Y-%m-%d')] * 3,
        'ARG_Element': ['Portal_1', 'Cipher_2', 'Theory_3']
    })
    
    print("Sample AI Oracle Script Data:")
    print(sample_data)
    print("\n⚠️  Note: Uncomment the code below to write to your sheet")
    
    # Uncomment to actually write:
    # sheet_id = os.getenv('GOOGLE_SHEET_ID', 'YOUR_SHEET_ID_HERE')
    # if sheet_id != 'YOUR_SHEET_ID_HERE':
    #     write_dataframe_to_sheet(client, sheet_id, sample_data, 'AI_Oracle_Scripts')
else:
    print("⚠️  Skipping write operation - client not authenticated")

## 5. Data Analysis with pandas

Demonstrate data manipulation and analysis techniques for AI Oracle project data.

In [ ]:
# Example analytics on script data
sample_analytics_data = pd.DataFrame({
    'Video_ID': ['VID001', 'VID002', 'VID003', 'VID004', 'VID005'],
    'Title': [
        'Oracle Message #1',
        'Decoding Reality',
        'The Hidden Truth',
        'Awakening Protocol',
        'Digital Prophecy'
    ],
    'Views': [15000, 22000, 18000, 12000, 25000],
    'Likes': [1200, 1800, 1500, 950, 2100],
    'Comments': [340, 520, 410, 280, 650],
    'Watch_Time_Minutes': [8500, 13200, 10800, 7200, 15000],
    'ARG_Engagement': [45, 68, 52, 38, 72]
})

# Calculate engagement metrics
sample_analytics_data['Engagement_Rate'] = (
    (sample_analytics_data['Likes'] + sample_analytics_data['Comments']) / 
    sample_analytics_data['Views'] * 100
).round(2)

sample_analytics_data['Avg_Watch_Time'] = (
    sample_analytics_data['Watch_Time_Minutes'] / sample_analytics_data['Views']
).round(2)

print("AI Oracle Video Analytics:")
print(sample_analytics_data)

print("\n📊 Summary Statistics:")
print(f"Total Views: {sample_analytics_data['Views'].sum():,}")
print(f"Average Engagement Rate: {sample_analytics_data['Engagement_Rate'].mean():.2f}%")
print(f"Top Performing Video: {sample_analytics_data.loc[sample_analytics_data['Views'].idxmax(), 'Title']}")
print(f"Average ARG Engagement: {sample_analytics_data['ARG_Engagement'].mean():.1f}")

## 6. Updating Specific Cells

How to update specific cells or ranges in Google Sheets.

In [ ]:
def update_cell_value(client, sheet_id, worksheet_name, cell_address, value):
    """
    Update a specific cell in a Google Sheet.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        worksheet_name: Name of the worksheet
        cell_address: Cell address (e.g., 'A1', 'B5')
        value: Value to set
    """
    try:
        spreadsheet = client.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)
        worksheet.update(cell_address, value)
        print(f"✅ Updated {cell_address} to '{value}'")
        return True
    except Exception as e:
        print(f"❌ Error updating cell: {e}")
        return False

def update_range(client, sheet_id, worksheet_name, range_address, values):
    """
    Update a range of cells in a Google Sheet.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        worksheet_name: Name of the worksheet
        range_address: Range address (e.g., 'A1:C3')
        values: 2D list of values
    """
    try:
        spreadsheet = client.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)
        worksheet.update(range_address, values)
        print(f"✅ Updated range {range_address}")
        return True
    except Exception as e:
        print(f"❌ Error updating range: {e}")
        return False

# Example usage (commented out to prevent accidental writes)
print("Example: Update status of a script")
print("update_cell_value(client, sheet_id, 'Scripts', 'C2', 'Published')")
print("\nExample: Update multiple cells at once")
print("update_range(client, sheet_id, 'Metrics', 'A1:C2', [['Date', 'Views', 'Engagement'], ['2024-01-15', '15000', '8.5%']])")

## 7. Best Practices and Error Handling

Key best practices when working with Google Sheets API:

In [ ]:
import time

def safe_batch_update(client, sheet_id, worksheet_name, data_list, batch_size=10):
    """
    Safely update Google Sheets in batches to avoid API rate limits.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        worksheet_name: Name of the worksheet
        data_list: List of data to append
        batch_size: Number of rows per batch
    """
    try:
        spreadsheet = client.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)
        
        total_rows = len(data_list)
        for i in range(0, total_rows, batch_size):
            batch = data_list[i:i+batch_size]
            worksheet.append_rows(batch)
            print(f"✅ Batch {i//batch_size + 1}: Added {len(batch)} rows")
            
            # Respect API rate limits (100 requests per 100 seconds)
            if i + batch_size < total_rows:
                time.sleep(1)
        
        print(f"✅ Successfully added {total_rows} rows in batches")
        return True
    
    except gspread.exceptions.APIError as e:
        print(f"❌ API Error: {e}")
        print("Tip: You may have hit rate limits. Try reducing batch size or adding delays.")
        return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

print("✅ Best Practices Tips:")
print("1. Always use try-except blocks for error handling")
print("2. Respect API rate limits (100 requests per 100 seconds)")
print("3. Use batch operations for multiple updates")
print("4. Store credentials securely in .env files")
print("5. Share sheets with service account email")
print("6. Use meaningful worksheet names for organization")
print("7. Implement logging for production use")

## 8. AI Oracle Specific Use Cases

Practical examples for the AI Oracle project workflow.

In [ ]:
def track_video_production(client, sheet_id, video_data):
    """
    Track video production status in Google Sheets.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        video_data: Dictionary with video information
    """
    df = pd.DataFrame([video_data])
    return write_dataframe_to_sheet(client, sheet_id, df, 'Production_Log', mode='append')

def log_arg_element(client, sheet_id, arg_data):
    """
    Log ARG (Alternate Reality Game) elements and clues.
    
    Args:
        client: Authenticated gspread client
        sheet_id: Google Sheet ID
        arg_data: Dictionary with ARG element information
    """
    df = pd.DataFrame([arg_data])
    return write_dataframe_to_sheet(client, sheet_id, df, 'ARG_Elements', mode='append')

# Example: Log a new video production
example_video = {
    'Date': datetime.now().strftime('%Y-%m-%d %H:%M'),
    'Video_Title': 'The Oracle Speaks: Message #42',
    'Script_Status': 'Complete',
    'Voiceover_Status': 'In Progress',
    'Video_Status': 'Not Started',
    'ARG_Layer': 'Cipher embedded in timestamp',
    'Target_Upload': '2024-02-01'
}

# Example: Log an ARG element
example_arg = {
    'Date': datetime.now().strftime('%Y-%m-%d'),
    'Element_ID': 'ARG_042',
    'Type': 'Hidden Message',
    'Location': 'Video description, line 3',
    'Clue': 'Base64 encoded message',
    'Solution': 'The truth is in the pattern',
    'Connected_Videos': 'VID001, VID015, VID042'
}

print("📹 Example Video Production Log:")
for key, value in example_video.items():
    print(f"  {key}: {value}")

print("\n🎮 Example ARG Element Log:")
for key, value in example_arg.items():
    print(f"  {key}: {value}")

print("\n⚠️  Uncomment the code below to log these to your sheet:")
# if client and os.getenv('GOOGLE_SHEET_ID'):
#     track_video_production(client, os.getenv('GOOGLE_SHEET_ID'), example_video)
#     log_arg_element(client, os.getenv('GOOGLE_SHEET_ID'), example_arg)

## Summary

This notebook demonstrated:
- ✅ Google Sheets API authentication using service accounts
- ✅ Reading data from sheets into pandas DataFrames
- ✅ Writing and updating data in Google Sheets
- ✅ Data analysis and manipulation with pandas
- ✅ Error handling and best practices
- ✅ AI Oracle specific use cases

### Next Steps:
1. Set up your Google Cloud project and service account
2. Download credentials and save to `credentials/google_sheets_credentials.json`
3. Create a Google Sheet for your AI Oracle project
4. Share the sheet with your service account email
5. Add `GOOGLE_SHEET_ID` to your `.env` file
6. Uncomment and run the example code blocks

### Resources:
- [gspread Documentation](https://docs.gspread.org/)
- [Google Sheets API Documentation](https://developers.google.com/sheets/api)
- [pandas Documentation](https://pandas.pydata.org/docs/)